In [57]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import geopy
from geopy.geocoders import Nominatim 
import folium
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

### Importing Libraries

# Part 1

In [3]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0]
df1 = pd.read_html(str(table))
df=pd.DataFrame(df1[0])

### Taking data from Wikipedia and converting into dataframe using pandas and beautiful soup

In [4]:
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Droping any NaN values

In [5]:
df.dropna( inplace=True)
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### Replacing the / with ,

In [6]:
df['Neighborhood'].replace(to_replace="/",value=",",regex=True,inplace=True)
df.head(10)

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern , Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill , Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [7]:
df.shape

(103, 3)

# Part 2

### Reading data from the given URL as geocoder library was not working.

In [8]:
df2=pd.read_csv('http://cocl.us/Geospatial_data')
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merging data on column Postal Code

In [9]:
df3= pd.merge(df, df2,left_on='Postal code',right_on='Postal Code')
df3.drop(labels=['Postal Code'],axis=1,inplace=True)
df3.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [10]:
df3.shape

(103, 5)

# Part 3

### Getting the Latitude and Longitude of Toronto using the Nominatim from geopy

In [25]:
address='Toronto,Ontario'
geoloc=Nominatim(user_agent='explorer')
location = geoloc.geocode(address)
longitude=location.longitude
latitude=location.latitude
print("The Data {},{}".format(latitude,longitude))

The Data 43.6534817,-79.3839347


#### A small map of Toronto marking Neighborhoods with a Small popup

In [27]:
maptoro = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df3['Latitude'],df3['Longitude'], df3['Borough'], df3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maptoro)  
    
maptoro

In [101]:
CLIENT_ID = 'B2KBZZZ1RTP5TJSWK124PV3SMUPIQZFLL4BHZ0X4UFKEVS2C' 
CLIENT_SECRET = 'J4ZZ5W2O2MC3WXRDY32N1L5HMXBVNCENQBSLEIFWBOEKYYHV' 
VERSION = '20200422'
limit=100
radius=500
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

CLIENT_ID: B2KBZZZ1RTP5TJSWK124PV3SMUPIQZFLL4BHZ0X4UFKEVS2C
CLIENT_SECRET:J4ZZ5W2O2MC3WXRDY32N1L5HMXBVNCENQBSLEIFWBOEKYYHV


#### Collecting data from Four Square using id, and requesting json data 

In [40]:
latitude = df3.loc[0, 'Latitude']
longitude =df3.loc[0, 'Longitude']

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=B2KBZZZ1RTP5TJSWK124PV3SMUPIQZFLL4BHZ0X4UFKEVS2C&client_secret=J4ZZ5W2O2MC3WXRDY32N1L5HMXBVNCENQBSLEIFWBOEKYYHV&v=20200422&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [42]:
results = requests.get(url).json()

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)  
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()    

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [46]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toro_venues = getNearbyVenues(names=df3['Neighborhood'],
                                   latitudes=df3['Latitude'],
                                   longitudes=df3['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park , Harbourfront
Lawrence Manor , Lawrence Heights
Queen's Park , Ontario Provincial Government
Islington Avenue
Malvern , Rouge
Don Mills
Parkview Hill , Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale
Rouge Hill , Port Union , Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood
Guildwood , Morningside , West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor , Wilson Heights , Downsview North
Thorncliffe Park
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Scarborough Village
Fairview , Henry Farm , Oriole
Northwood Park , York University
East Toronto
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
Kennedy Park , Ionview , East Birchmount Park
Bayview Village
Do

### Function to get data from Four Square for all the Neighborhood

In [47]:
print(toro_venues.shape)
toro_venues.head()

(2143, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


## Analyzing the each data

In [49]:
toro_onehot = pd.get_dummies(toro_venues[['Venue Category']], prefix="", prefix_sep="")
toro_onehot['Neighborhood'] = toro_venues['Neighborhood'] 
fixed_columns = [toro_onehot.columns[-1]] + list(toro_onehot.columns[:-1])
toro_onehot =toro_onehot[fixed_columns]

toro_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
toro_onehot.shape

(2143, 269)

In [51]:
toro_grouped = toro_onehot.groupby('Neighborhood').mean().reset_index()
toro_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,"Alderwood , Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,"Bathurst Manor , Wilson Heights , Downsview North",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.0,0.000000
3,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,"Bedford Park , Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.043478
5,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,"Birch Cliff , Cliffside West",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,"Brockton , Parkdale Village , Exhibition Place",0.043478,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,Business reply mail Processing CentrE,0.058824,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,"CN Tower , King and Spadina , Railway Lands , ...",0.000000,0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [52]:
toro_grouped.shape

(94, 269)

In [54]:
num_top_venues = 5

for hood in toro_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toro_grouped[toro_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1               Skating Rink  0.25
2             Breakfast Spot  0.25
3  Latin American Restaurant  0.25
4              Metro Station  0.00


----Alderwood , Long Branch----
          venue  freq
0   Pizza Place  0.22
1          Pool  0.11
2  Skating Rink  0.11
3           Gym  0.11
4      Pharmacy  0.11


----Bathurst Manor , Wilson Heights , Downsview North----
            venue  freq
0            Bank  0.10
1     Coffee Shop  0.10
2       Gift Shop  0.05
3   Shopping Mall  0.05
4  Sandwich Place  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3  Japanese Restaurant  0.25
4          Yoga Studio  0.00


----Bedford Park , Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.09
1         Coffee Shop  0.09
2          Restaurant  0.09
3      Sandwich Place  0.09
4       Women's S

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Getting top 10 venues of each neighborhood

In [76]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Neighborhood'] = toro_grouped['Neighborhood']

for ind in np.arange(toro_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(toro_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Breakfast Spot,Women's Store,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood , Long Branch",Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Pub,Pool,Skating Rink,Gym,Construction & Landscaping,Department Store
2,"Bathurst Manor , Wilson Heights , Downsview North",Bank,Coffee Shop,Sushi Restaurant,Bridal Shop,Diner,Restaurant,Fried Chicken Joint,Pizza Place,Pharmacy,Deli / Bodega
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Women's Store,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
4,"Bedford Park , Lawrence Manor East",Sandwich Place,Italian Restaurant,Restaurant,Coffee Shop,Women's Store,Thai Restaurant,Comfort Food Restaurant,Juice Bar,Butcher,Café


# Clustering

In [77]:
k = 5
torocluster = toro_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=k, random_state=0).fit(torocluster)
kmeans.labels_[0:10] 
df3.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [78]:
venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_.astype(int))

toro_merged = df3

toro_merged = toro_merged.join(venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toro_merged.head() 

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,Food & Drink Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Falafel Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,Pizza Place,Portuguese Restaurant,Hockey Arena,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Theater,Café,Health Food Store,Historic Site,Hotel
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Women's Store,Furniture / Home Store,Coffee Shop,Boutique,Shoe Store,Miscellaneous Shop,Event Space,Vietnamese Restaurant,Accessories Store
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Creperie,Spa,Beer Bar,Sandwich Place,Burger Joint,Burrito Place


### The above cells are used to create clusters and the below cell is used to create a map out of these cluster.

In [100]:
# create map
maptoro_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(toro_merged['Latitude'], toro_merged['Longitude'], toro_merged['Neighborhood'],toro_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    i=i+1
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(clusters-1)],
        fill=True,
        fill_color=rainbow[int(clusters-1)],
        fill_opacity=0.7).add_to(maptoro_clusters)
        
       
maptoro_clusters

### Indepently shown Cluster 1,2,3,4,5 respectively in each cell

In [85]:
toro_merged.loc[toro_merged['Cluster Labels'] == 0,toro_merged.columns[[1] + list(range(5, toro_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Coffee Shop,Pizza Place,Portuguese Restaurant,Hockey Arena,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
22,Scarborough,0.0,Coffee Shop,Korean Restaurant,Indian Restaurant,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
50,North York,0.0,Empanada Restaurant,Pizza Place,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Drugstore
70,Etobicoke,0.0,Pizza Place,Intersection,Discount Store,Coffee Shop,Sandwich Place,Chinese Restaurant,Women's Store,Dog Run,Diner,Distribution Center
77,Etobicoke,0.0,Bus Line,Pizza Place,Sandwich Place,Mobile Phone Shop,Women's Store,Distribution Center,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
93,Etobicoke,0.0,Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Pub,Pool,Skating Rink,Gym,Construction & Landscaping,Department Store


In [92]:
toro_merged.loc[toro_merged['Cluster Labels'] == 1,toro_merged.columns[[1] + list(range(5, toro_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1.0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Theater,Café,Health Food Store,Historic Site,Hotel
3,North York,1.0,Clothing Store,Women's Store,Furniture / Home Store,Coffee Shop,Boutique,Shoe Store,Miscellaneous Shop,Event Space,Vietnamese Restaurant,Accessories Store
4,Downtown Toronto,1.0,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Creperie,Spa,Beer Bar,Sandwich Place,Burger Joint,Burrito Place
6,Scarborough,1.0,Fast Food Restaurant,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
7,North York,1.0,Coffee Shop,Gym,Asian Restaurant,Beer Store,Japanese Restaurant,Restaurant,Gym / Fitness Center,Supermarket,Discount Store,Café
8,East York,1.0,Pizza Place,Gym / Fitness Center,Pharmacy,Intersection,Athletics & Sports,Bus Line,Bank,Gastropub,Pet Store,Fast Food Restaurant
9,Downtown Toronto,1.0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Fast Food Restaurant,Italian Restaurant
10,North York,1.0,Japanese Restaurant,Park,Pizza Place,Pub,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
12,Scarborough,1.0,Construction & Landscaping,Bar,Women's Store,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
13,North York,1.0,Coffee Shop,Gym,Asian Restaurant,Beer Store,Japanese Restaurant,Restaurant,Gym / Fitness Center,Supermarket,Discount Store,Café


In [93]:
toro_merged.loc[toro_merged['Cluster Labels'] == 2,toro_merged.columns[[1] + list(range(5, toro_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2.0,Park,Food & Drink Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Falafel Restaurant
16,York,2.0,Park,Field,Hockey Arena,Trail,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Department Store
21,York,2.0,Park,Pool,Women's Store,Airport,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
35,East York,2.0,Park,Coffee Shop,Convenience Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant
61,Central Toronto,2.0,Park,Swim School,Bus Line,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Falafel Restaurant
64,York,2.0,Park,Convenience Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
66,North York,2.0,Park,Bank,Convenience Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
68,Central Toronto,2.0,Trail,Park,Sushi Restaurant,Jewelry Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
83,Central Toronto,2.0,Park,Playground,Trail,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Empanada Restaurant,Donut Shop,Deli / Bodega
85,Scarborough,2.0,Park,Coffee Shop,Playground,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


In [95]:
toro_merged.loc[toro_merged['Cluster Labels'] == 3,toro_merged.columns[[1] + list(range(5, toro_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,3.0,Martial Arts Dojo,Women's Store,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop


In [96]:
toro_merged.loc[toro_merged['Cluster Labels'] == 4,toro_merged.columns[[1] + list(range(5, toro_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,4.0,Food Service,Baseball Field,Women's Store,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Farmers Market
101,Etobicoke,4.0,Locksmith,Baseball Field,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
